# Cleaning the data
We need:

A article data,
K knab data

1.   A- URLS without duplicates and all corresponding titles
2.   A- Not clean URLS, with duplicates (articles)
3.   A- Clean article text data (no duplicates)
4.   K- Cleaned page_data
5.   K- Cleaned event_data

In [0]:
import glob
import pandas as pd
from google.colab import files
from google.colab import drive
import numpy as np
from tqdm import tqdm
from collections import Counter
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

In [0]:
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Knab/Data/Knab_Analytics/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 architecture  'https:  surfdrive.gdoc'  'Untitled Diagram.xml'
 DataEvents     merged_data.csv


##Import all data

In [0]:
#Load in all the PAGE files
page_files = glob.glob("/content/drive/My Drive/Knab/Data/Knab_Analytics/DataEvents/Pages/*.csv")
#print(page_files)
page_dfs = [pd.read_csv(p, sep=",") for p in page_files]
page_data = pd.concat(page_dfs,ignore_index=False)
page_data.name = 'Page_Data'

In [0]:
#Load in all the EVENT files
event_files = glob.glob("/content/drive/My Drive/Knab/Data/Knab_Analytics/DataEvents/Events/*.csv")
#print(event_files)
event_dfs = [pd.read_csv(e, sep=",") for e in event_files]
event_data = pd.concat(event_dfs,ignore_index=False)
event_data.name = 'Event_Data'

In [0]:
#Load in all urls with tags and text
df_urlstext = pd.read_excel("/content/drive/My Drive/Knab/Data/Articles/Scraped/URLTEXT.xlsx")

In [0]:
#Split dfs (remove the text for memory space)
df_urls = df_urlstext[['URL','TAG','TITLE','DATE','READING_TIME']]

##Clean the url data

In [0]:
#Change data types
df_urls['URL'] = df_urls['URL'].astype('str')
df_urls['TAG'] = df_urls['TAG'].astype('str')
df_urls['TITLE'] = df_urls['TITLE'].astype('str')
df_urls['READING_TIME'] = df_urls['READING_TIME'].astype('str')
df_urls['DATE'] = pd.to_datetime(df_urls['DATE'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

## Lookup table

In [0]:
#Make table of all urls and add empty column
lookup_urls = df_urls['URL'].values
lookup_urls = np.vstack((np.zeros(len(lookup_urls)),lookup_urls)).transpose()

In [0]:
#Fill empty column with correct urls
duplicate_table = pd.concat(g for _, g in df_urls.groupby("TITLE") if len(g) > 1) #Find all duplicates

In [0]:
for i in range(len(lookup_urls)):
  if lookup_urls[:,1][i] not in duplicate_table['URL'].values:
    lookup_urls[i,0] = lookup_urls[i,1]
  else:
    #Find title of duplicated url and find all the other corresponding urls
    title = duplicate_table[duplicate_table['URL'] == lookup_urls[:,1][i]]['TITLE']
    dup_urls = duplicate_table[duplicate_table['TITLE'] == title.values[0]]['URL']
    true_url = dup_urls.values[0] #Take first url in the list
    lookup_urls[i,0] = true_url

In [0]:
#Change to dataframe
lookup_df = pd.DataFrame(lookup_urls)
lookup_df.columns = ['URL', 'Reference']
lookup_df.to_csv('lookup_urls.csv', index=False)

## Article data

In [0]:
#Drop all the duplicate urls
no_duplicates = lookup_df.drop_duplicates(['URL'])

In [0]:
#Merge with other information
clean_article_data = pd.merge(no_duplicates, df_urlstext, left_on='URL', right_on='URL',how='left').drop(['Reference'], axis=1)
clean_article_data.to_csv('clean_article_data.csv', index=False)

# Clean the Knab data

In [0]:
def remove_nan(all_data):
  #Remove NaN column
  all_data = all_data.loc[:, all_data.columns.notnull()]

  #Drop all NaN rows
  all_data = all_data.dropna()
  
  return all_data

In [0]:
#Drop index column and reindex
page_data = page_data.drop(['Unnamed: 0'], axis = 1)
page_data = page_data.reset_index(drop=True)

event_data = event_data.drop(['Unnamed: 0'], axis = 1)
event_data = event_data.reset_index(drop=True)

In [0]:
#Drop NaNs
page_data = remove_nan(page_data)
event_data = remove_nan(event_data)

In [0]:
#Change data types
#Integers
page_data['visitid'] = page_data['visitid'].astype('int64')
page_data['hitnumber'] = page_data['hitnumber'].astype('int64')
page_data['time'] = page_data['time'].astype('int64')
page_data['BiebYN'] = page_data['BiebYN'].astype('int64')

event_data['visitid'] = event_data['visitid'].astype('int64')
event_data['hitnumber'] = event_data['hitnumber'].astype('int64')
event_data['BiebYN'] = event_data['BiebYN'].astype('int64')

#Strings
page_data['clientid_hashed'] = page_data['clientid_hashed'].astype('str')
page_data['pagepath'] = page_data['pagepath'].astype('str')
page_data['channelgrouping'] = page_data['channelgrouping'].astype('str')
page_data['browser'] = page_data['browser'].astype('str')
page_data['devicecategory'] = page_data['devicecategory'].astype('str')

event_data['clientid_hashed'] = event_data['clientid_hashed'].astype('str')
event_data['pagepath'] = event_data['pagepath'].astype('str')
event_data['channelgrouping'] = event_data['channelgrouping'].astype('str')
event_data['browser'] = event_data['browser'].astype('str')
event_data['devicecategory'] = event_data['devicecategory'].astype('str')
event_data['eventcategory'] = event_data['eventcategory'].astype('str')
event_data['eventlabel'] = event_data['eventlabel'].astype('str')
event_data['ZakPart'] = event_data['ZakPart'].astype('str')


#Datetime
page_data['visitstarttime'] = pd.to_datetime(page_data['visitstarttime'])
event_data['visitstarttime'] = pd.to_datetime(event_data['visitstarttime'])

## Clean page data

In [0]:
#Replace the url duplicates with correct ones from lookup table
merge1 = pd.merge(page_data, lookup_df, left_on = 'pagepath', right_on='Reference', how='left')
merge1['URL'].fillna(merge1['pagepath'], inplace = True)

In [0]:
#Remove the wrong url columns and reorder
clean_page_data = merge1[['URL', 'clientid_hashed','visitid','visitstarttime','hitnumber','time','channelgrouping','browser','devicecategory','BiebYN']]

In [0]:
#Save to csv
clean_page_data.to_csv('clean_page_data.csv', index=False)

##Clean event data


In [0]:
#Replace the url duplicates with correct ones from lookup table
merge2 = pd.merge(event_data, lookup_df, left_on = 'pagepath', right_on='Reference', how='left')
merge2['URL'].fillna(merge2['pagepath'], inplace = True)

In [0]:
#Remove the wrong url columns and reorder
clean_event_data = merge2[['URL', 'clientid_hashed','visitid','visitstarttime','hitnumber','eventcategory','eventlabel','ZakPart']]

In [0]:
#Save to csv
clean_event_data.to_csv('clean_event_data.csv', index=False)